# Mitandao ya Kizazi

Mitandao ya Neural ya Kurudiarudia (RNNs) na aina zake zenye seli zenye milango kama vile Long Short Term Memory Cells (LSTMs) na Gated Recurrent Units (GRUs) zilitoa njia ya kuunda mifano ya lugha, yaani, zinaweza kujifunza mpangilio wa maneno na kutoa utabiri wa neno linalofuata katika mfuatano. Hii inatuwezesha kutumia RNNs kwa **kazi za kizazi**, kama vile utengenezaji wa maandishi ya kawaida, tafsiri ya mashine, na hata uundaji wa maelezo ya picha.

Katika usanifu wa RNN tuliojadili katika sehemu iliyopita, kila kitengo cha RNN kilizalisha hali iliyofichwa inayofuata kama matokeo. Hata hivyo, tunaweza pia kuongeza matokeo mengine kwa kila kitengo cha kurudiarudia, ambacho kingeturuhusu kutoa **mfuatano** (ambao ni sawa kwa urefu na mfuatano wa awali). Zaidi ya hayo, tunaweza kutumia vitengo vya RNN ambavyo havipokei ingizo katika kila hatua, na badala yake huchukua tu hali ya awali kama vekta, na kisha huzalisha mfuatano wa matokeo.

Katika daftari hili, tutazingatia mifano rahisi ya kizazi inayotusaidia kuzalisha maandishi. Kwa urahisi, hebu tujenge **mtandao wa kiwango cha herufi**, ambao huzalisha maandishi herufi moja moja. Wakati wa mafunzo, tunahitaji kuchukua mkusanyiko wa maandishi, na kuugawanya katika mfuatano wa herufi.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Kujenga Msamiati wa Herufi

Ili kujenga mtandao wa kizazi wa kiwango cha herufi, tunahitaji kugawanya maandishi kuwa herufi moja-moja badala ya maneno. Safu ya `TextVectorization` ambayo tumekuwa tukitumia hapo awali haiwezi kufanya hivyo, kwa hivyo tuna chaguzi mbili:

* Kupakia maandishi kwa mikono na kufanya tokenization 'kwa mkono', kama ilivyo katika [mfano huu rasmi wa Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Kutumia darasa la `Tokenizer` kwa tokenization ya kiwango cha herufi.

Tutachagua chaguo la pili. `Tokenizer` pia inaweza kutumika kugawanya katika maneno, kwa hivyo mtu anaweza kubadilisha kwa urahisi kutoka tokenization ya kiwango cha herufi hadi kiwango cha maneno.

Ili kufanya tokenization ya kiwango cha herufi, tunahitaji kupitisha parameter `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Tunataka pia kutumia token moja maalum kuashiria **mwisho wa mfuatano**, ambayo tutaiita `<eos>`. Wacha tuiongeze kwa mkono kwenye msamiati:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Kufundisha RNN ya kizazi kutengeneza vichwa vya habari

Njia tutakayotumia kufundisha RNN ili kutengeneza vichwa vya habari ni kama ifuatavyo. Kwenye kila hatua, tutachukua kichwa kimoja, ambacho kitaingizwa kwenye RNN, na kwa kila herufi ya ingizo tutaiomba mtandao kutengeneza herufi inayofuata ya matokeo:

![Picha inayoonyesha mfano wa kizazi cha RNN cha neno 'HELLO'.](../../../../../translated_images/sw/rnn-generate.56c54afb52f9781d.webp)

Kwa herufi ya mwisho ya mlolongo wetu, tutaiomba mtandao kutengeneza tokeni `<eos>`.

Tofauti kuu ya RNN ya kizazi tunayotumia hapa ni kwamba tutachukua matokeo kutoka kila hatua ya RNN, na siyo tu kutoka kwenye seli ya mwisho. Hii inaweza kufanikishwa kwa kuweka kipengele `return_sequences` kwenye seli ya RNN.

Kwa hivyo, wakati wa mafunzo, ingizo kwa mtandao litakuwa mlolongo wa herufi zilizofichwa wa urefu fulani, na matokeo yatakuwa mlolongo wa urefu huo huo, lakini uliosogezwa kwa kipengele kimoja na kumalizika na `<eos>`. Kundi dogo (minibatch) litajumuisha milolongo kadhaa kama hiyo, na tutahitaji kutumia **padding** ili kulinganisha milolongo yote.

Hebu tuunde kazi zitakazobadilisha seti ya data kwa ajili yetu. Kwa sababu tunataka kuweka padding kwenye kiwango cha minibatch, tutaanza kwa kugawanya seti ya data kwa kupiga `.batch()`, na kisha kutumia `map` ili kufanya mabadiliko. Kwa hivyo, kazi ya mabadiliko itachukua minibatch nzima kama parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Mambo muhimu tunayofanya hapa:
* Kwanza tunatoa maandishi halisi kutoka kwa tensor ya maandishi
* `text_to_sequences` hubadilisha orodha ya maandishi kuwa orodha ya tensors za nambari
* `pad_sequences` huongeza nafasi kwenye tensors hizo hadi urefu wake wa juu
* Hatimaye tunafanya one-hot encoding kwa herufi zote, na pia tunafanya mabadiliko na kuongeza `<eos>`. Tutakuja kuona hivi karibuni kwa nini tunahitaji herufi zilizo katika one-hot encoding.

Hata hivyo, kazi hii ni **Pythonic**, yaani haiwezi kutafsiriwa moja kwa moja kuwa grafu ya kihesabu ya Tensorflow. Tutapata makosa tukijaribu kutumia kazi hii moja kwa moja katika kazi ya `Dataset.map`. Tunahitaji kufunika mwito huu wa Pythonic kwa kutumia kifuniko cha `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Kumbuka**: Kutofautisha kati ya kazi za mabadiliko za Pythonic na Tensorflow kunaweza kuonekana kuwa ngumu kidogo, na unaweza kujiuliza kwa nini hatubadilishi dataset kwa kutumia kazi za kawaida za Python kabla ya kuipitisha kwa `fit`. Ingawa hili linaweza kufanyika, kutumia `Dataset.map` kuna faida kubwa, kwa sababu mchakato wa mabadiliko ya data unatekelezwa kwa kutumia grafu ya kihesabu ya Tensorflow, ambayo inachukua faida ya mahesabu ya GPU, na inapunguza hitaji la kupitisha data kati ya CPU/GPU.

Sasa tunaweza kujenga mtandao wetu wa jenereta na kuanza mafunzo. Unaweza kutegemea seli yoyote ya kurudia tuliyojadili katika kitengo kilichopita (rahisi, LSTM au GRU). Katika mfano wetu tutatumia LSTM.

Kwa sababu mtandao unachukua herufi kama pembejeo, na ukubwa wa msamiati ni mdogo sana, hatuhitaji safu ya embedding, pembejeo iliyowakilishwa kwa njia ya one-hot inaweza kwenda moja kwa moja kwenye seli ya LSTM. Safu ya pato itakuwa `Dense` classifier ambayo itabadilisha pato la LSTM kuwa namba za tokeni zilizowakilishwa kwa njia ya one-hot.

Zaidi ya hayo, kwa kuwa tunashughulika na misururu ya urefu tofauti, tunaweza kutumia safu ya `Masking` kuunda maski ambayo itapuuza sehemu ya mfuatano iliyojazwa. Hii si lazima kabisa, kwa sababu hatuvutiwi sana na kila kitu kinachozidi tokeni `<eos>`, lakini tutaifanya kwa lengo la kupata uzoefu na aina hii ya safu. `input_shape` itakuwa `(None, vocab_size)`, ambapo `None` inaonyesha mfuatano wa urefu tofauti, na umbo la pato ni `(None, vocab_size)` pia, kama unavyoweza kuona kutoka kwa `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Kuzalisha matokeo

Sasa kwamba tumefundisha modeli, tunataka kuitumia kuzalisha matokeo fulani. Kwanza kabisa, tunahitaji njia ya kutafsiri maandishi yanayowakilishwa na mfululizo wa namba za tokeni. Ili kufanya hivyo, tunaweza kutumia kazi ya `tokenizer.sequences_to_texts`; hata hivyo, haifanyi kazi vizuri na tokeni za kiwango cha herufi. Kwa hivyo tutachukua kamusi ya tokeni kutoka kwa tokenizer (inayoitwa `word_index`), kujenga ramani ya kurudi nyuma, na kuandika kazi yetu ya kutafsiri:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Sasa, hebu tufanye kizazi. Tutaanza na kamba fulani `start`, tuitafsiri kuwa mlolongo `inp`, na kisha katika kila hatua tutaita mtandao wetu ili kutabiri herufi inayofuata.

Matokeo ya mtandao `out` ni vector yenye vipengele `vocab_size` vinavyowakilisha uwezekano wa kila tokeni, na tunaweza kupata namba ya tokeni yenye uwezekano mkubwa zaidi kwa kutumia `argmax`. Kisha tunaongeza herufi hii kwenye orodha ya tokeni zilizotengenezwa, na tunaendelea na kizazi. Mchakato huu wa kutengeneza herufi moja unarudiwa mara `size` ili kuzalisha idadi inayohitajika ya herufi, na tunakomesha mapema ikiwa `eos_token` itapatikana.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Kuchukua sampuli ya matokeo wakati wa mafunzo

Kwa sababu hatuna vipimo vyovyote vya maana kama *usahihi*, njia pekee ya kuona kwamba modeli yetu inaboreka ni kwa **kuchukua sampuli** ya mfululizo wa maandishi yanayozalishwa wakati wa mafunzo. Ili kufanya hivyo, tutatumia **callbacks**, yaani, kazi ambazo tunaweza kupitisha kwa `fit` function, na ambazo zitaitwa mara kwa mara wakati wa mafunzo.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Mfano huu tayari unazalisha maandishi mazuri, lakini unaweza kuboreshwa zaidi kwa njia kadhaa:

* **Maandishi zaidi**. Tumetumia tu vichwa vya habari kwa kazi yetu, lakini unaweza kutaka kujaribu na maandishi kamili. Kumbuka kwamba RNN hazifanyi kazi vizuri sana na mfuatano mrefu, kwa hivyo inafaa kugawanya katika sentensi fupi, au kila wakati kufundisha kwa urefu wa mfuatano uliowekwa wa thamani fulani iliyotanguliwa `num_chars` (kwa mfano, 256). Unaweza kujaribu kubadilisha mfano hapo juu kuwa usanifu kama huo, ukitumia [mafunzo rasmi ya Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) kama msukumo.

* **LSTM ya tabaka nyingi**. Inafaa kujaribu tabaka 2 au 3 za seli za LSTM. Kama tulivyotaja katika kitengo kilichopita, kila tabaka la LSTM huchota mifumo fulani kutoka kwa maandishi, na kwa kizazi cha kiwango cha herufi tunaweza kutarajia tabaka za chini za LSTM kushughulikia silabi, na tabaka za juu - maneno na mchanganyiko wa maneno. Hii inaweza kutekelezwa kwa urahisi kwa kupitisha kipengele cha idadi-ya-tabaka kwa mjenzi wa LSTM.

* Unaweza pia kutaka kujaribu na **vitengo vya GRU** na kuona ni vipi vinafanya kazi bora, na pia na **saizi tofauti za tabaka zilizofichwa**. Tabaka kubwa sana iliyofichwa inaweza kusababisha kujifunza kupita kiasi (kwa mfano, mtandao utajifunza maandishi halisi), na saizi ndogo inaweza isizalishe matokeo mazuri.


## Uzalishaji wa maandishi laini na joto

Katika ufafanuzi wa awali wa `generate`, tulikuwa tunachukua herufi yenye uwezekano wa juu zaidi kama herufi inayofuata katika maandishi yanayozalishwa. Hii ilisababisha maandishi mara nyingi "kurudia" mfululizo wa herufi zile zile tena na tena, kama katika mfano huu:
```
today of the second the company and a second the company ...
```

Hata hivyo, tukitazama usambazaji wa uwezekano kwa herufi inayofuata, inaweza kuwa tofauti kati ya uwezekano wa juu zaidi si kubwa sana, kwa mfano herufi moja inaweza kuwa na uwezekano wa 0.2, nyingine - 0.19, nk. Kwa mfano, tunapotafuta herufi inayofuata katika mfululizo '*play*', herufi inayofuata inaweza kuwa nafasi, au **e** (kama katika neno *player*).

Hii inatupeleka kwenye hitimisho kwamba si kila wakati ni "haki" kuchagua herufi yenye uwezekano wa juu zaidi, kwa sababu kuchagua ya pili yenye uwezekano wa juu bado inaweza kutupeleka kwenye maandishi yenye maana. Ni busara zaidi **kuchagua kwa sampuli** herufi kutoka kwa usambazaji wa uwezekano uliotolewa na matokeo ya mtandao.

Sampuli hii inaweza kufanywa kwa kutumia kazi ya `np.multinomial` ambayo inatekeleza kinachoitwa **usambazaji wa multinomial**. Kazi inayotekeleza uzalishaji huu wa maandishi **laini** imefafanuliwa hapa chini:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Tumetambulisha kipengele kingine kinachoitwa **joto**, ambacho kinatumika kuonyesha jinsi tunavyopaswa kushikilia kwa nguvu uwezekano wa juu zaidi. Ikiwa joto ni 1.0, tunafanya sampuli ya haki ya multinomial, na wakati joto linaenda hadi ukomo - uwezekano wote unakuwa sawa, na tunachagua herufi inayofuata kwa nasibu. Katika mfano hapa chini tunaweza kuona kwamba maandishi yanakuwa hayana maana tunapoongeza joto kupita kiasi, na yanakuwa kama maandishi magumu yaliyo "zungushwa" tunapokaribia 0.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kwa usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, inashauriwa kutumia tafsiri ya kitaalamu ya binadamu. Hatutawajibika kwa maelewano mabaya au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
